In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import pandas as pd
from collections import defaultdict
import hashlib, random, math, bisect
import os
from collections import defaultdict as _dd
import random as _rnd

# Constants 

In [2]:
SLOTS_PER_EPOCH = 2400
SLOT_DURATION_SEC = 30
EEP_FRACTION = 1/3
EEP_SLOT = int(SLOTS_PER_EPOCH * EEP_FRACTION)
PENALTY_FACTOR = 2.0
RECOVERY_FACTOR = 1.0
TOLERATED_BAD_RATIO = 0.02
REWARD_PER_BLOCK = 0.0001
AUTO_COMPOUND = True
RNG_SEED = 42
MIN_BET_AMOUNT = 50.0
TOKEN_SCALE_INT = 10**18
W_SKIPPED = 1.0
W_INVALID = 2.0
W_DELAYED = 1.5

# Classes

In [3]:
@dataclass
class Wallet:
    address: str
    green: float = 0.0
    red: float   = 0.0

    def get_balance(self) -> Dict[str, float]:
        """Ritorna i saldi correnti (green=stake, red=fee)."""
        return {"green": self.green, "red": self.red}

    def deposit(self, green: float = 0.0, red: float = 0.0) -> None:
        """Ricarica bilanci (stake/fee)."""
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        """
        Trasferisce token verso un altro wallet (come nel wallet reale: indichi green e red).
        Metti 0 se vuoi trasferire solo uno dei due.
        """
        if green_amount < 0 or red_amount < 0:
            return False
        if green_amount > self.green or red_amount > self.red:
            return False

        if green_amount:
            self.green -= green_amount
            other.green += green_amount
        if red_amount:
            self.red -= red_amount
            other.red += red_amount
        return (green_amount > 0) or (red_amount > 0)

    def pay_fee(self, amount: float) -> bool:
        """
        Paga una fee consumando prima RED (TKR) e solo se insufficiente usa GREEN (TKG).
        """
        if amount <= 0:
            return True

        use_red = min(self.red, amount)
        self.red -= use_red
        remaining = amount - use_red
        if remaining <= 0:
            return True

        if remaining > self.green:
            self.green = max(0.0, self.green - remaining)
            return False
        self.green -= remaining
        return True

In [4]:
@dataclass
class Stakeholder(Wallet):
    display_name: Optional[str] = None

    def bet_op(self, to_main_id: str, amount: float, when_slot: int) -> dict:
        return {"kind": "STAKE", "from": self.address, "to": to_main_id,
                "amount": float(amount), "slot": when_slot}

In [5]:
@dataclass
class MainAddress(Wallet):
    delegates: Dict[str, float] = field(default_factory=dict)

    active_stake: float = 0.0

    overflows: Dict[str, "OverflowNode"] = field(default_factory=dict)

    def register_overflow(self, overflow: "OverflowNode") -> None:
        """
        Registra un overflow associato a questo main.
        Lo aggiunge al dizionario 'overflows' e aggiorna il main_id del nodo.
        """
        overflow.main_id = self.address
        self.overflows[overflow.address] = overflow

    def get_overflow_list(self) -> list:
        """Ritorna la lista degli overflow associati a questo main."""
        return list(self.overflows.values())

    def bet(self, from_address: str, amount: float) -> bool:
        """
        Riceve una delega (bet) in GREEN da uno stakeholder o dal main stesso.
        La quantità deve superare MIN_BET_AMOUNT.
        """
        if amount < MIN_BET_AMOUNT:
            return False
        self.delegates[from_address] = self.delegates.get(from_address, 0.0) + amount
        return True

    def self_bet(self, amount: float) -> bool:
        """
        Il main punta su sé stesso (auto-stake).
        Scala i suoi green e registra la delega come '_self_'.
        """
        if amount < MIN_BET_AMOUNT or amount > self.green:
            return False
        self.green -= amount
        return self.bet("_self_", amount)

    def self_unbet(self, amount: Optional[float] = None) -> float:
        """
        Revoca la self-bet del MAIN (tutta o parziale).
        Sposta dalla voce delegates['_self_'] ai green del main.
        :param amount: None => ritira TUTTA la self-bet; altrimenti ritira 'amount' (cappato al disponibile).
        :return: quantità effettivamente ritirata.
        """
        cur = self.delegates.get("_self_", 0.0)
        if cur <= 0:
            return 0.0
        take = cur if amount is None else max(0.0, min(amount, cur))
        if take <= 0:
            return 0.0

        residual = cur - take
        if residual > 0:
            self.delegates["_self_"] = residual
        else:
            self.delegates.pop("_self_", None)

        self.green += take
        return take

    def total_delegated_now(self) -> float:
        """Somma di tutte le deleghe correnti (inclusi self-bet e reward composte)."""
        return sum(self.delegates.values())

    def snapshot_at_eep(self) -> None:
        """Congela la stake attiva per l’epoch successivo."""
        self.active_stake = self.total_delegated_now()

    def credit_reward(self, amount: float) -> None:
        """
        Distribuisce la reward proporzionalmente alle deleghe correnti,
        componendola subito nella stake di ciascun delegante.
        """
        if amount <= 0:
            return
        total = self.total_delegated_now()
        if total <= 0:
            self.green += amount
            return

        for addr, stake_amt in list(self.delegates.items()):
            share = amount * (stake_amt / total)
            self.delegates[addr] = stake_amt + share

In [6]:
@dataclass
class OverflowNode(Wallet):
    main_id: str = ""
    base_stake_share: float = 1.0

    penalty_score: float = 0.0
    p_skip: float = 0.00
    p_invalid: float = 0.00
    p_delayed: float = 0.00

    effective_stake: float = 0.0
    assigned_slots: int = 0
    penalty_slots_residual: int = 0

    def behavior_outcome(self, rng) -> str:
        """Esito di uno slot (stocastico) in base ai parametri di affidabilità."""
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        """
        Applica una penalità sul COLLATERAL (GREEN) del nodo.
        Ritorna quanto è stato effettivamente tagliato.
        """
        if amount <= 0:
            return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [ ]:
class TakamakaSim:
    """
    Simulatore aderente alla logica della chain, **senza finalità**.

    Fix precisione stake:
      - Pesi VRF quantizzati in **interi** con scala grande (TOKEN_SCALE_INT) e **troncamento** (no round).
      - Mapping ticket → intervallo con **bisect_left** (i bordi appartengono all’intervallo precedente).

    Temporalità:
      - Seed fissato all'EEP (1/3) dall'ultimo blocco valido nel primo terzo (back-search se saltato).
      - Intervalli stake-proporzionali ordinati per INT_ADDR = Hash(ADDR).
      - Estrazione per-slot via hash iterato del seed (PRF deterministica).
      - NEXT calcolata dopo l'EEP dell'epoca corrente.
      - Bootstrap epoca 0: CESD0 via VRF e NESD0 = CESD0.

    Penalità/Reward:
      - Reward = coinbase * (#OK).
      - Penalità in **penalty slots** pesando gli errori (invalid > skipped), con **recovery per slot**:
        penalty_resid_next = max(0, resid - RECOVERY_FACTOR*OK) + ceil(PENALTY_FACTOR * max(0, bad_weighted - tol))
      - La **stake efficace** dell’epoch successiva si ottiene attenuando la base con il residuo:
        eff = base / (1 + penalty_alpha * penalty_slots_residual)
    """

    def __init__(self,
                 *,
                 slots_per_epoch: int = SLOTS_PER_EPOCH,
                 slot_duration_sec: int = SLOT_DURATION_SEC,
                 eep_fraction: float = EEP_FRACTION,
                 penalty_factor: float = PENALTY_FACTOR,
                 recovery_factor: float = RECOVERY_FACTOR,
                 tolerated_bad_ratio: float = TOLERATED_BAD_RATIO,
                 reward_per_block: float = REWARD_PER_BLOCK,
                 seed: int = RNG_SEED,
                 slash_per_invalid: float = 0.0,
                 penalty_alpha: float = 1e-3,
                 w_skipped: float = 1.0,
                 w_invalid: float = 2.0,
                 w_delayed: float = 1.5):

        self.cfg = {
            "slots_per_epoch": int(slots_per_epoch),
            "slot_duration_sec": int(slot_duration_sec),
            "eep_slot": int(slots_per_epoch * eep_fraction),
            "penalty_factor": float(penalty_factor),
            "recovery_factor": float(recovery_factor),
            "tolerated_bad_ratio": float(tolerated_bad_ratio),
            "reward_per_block": float(reward_per_block),
            "seed": int(seed),
            "slash_per_invalid": float(slash_per_invalid),
            "stake_scale_int": int(TOKEN_SCALE_INT),
            "penalty_alpha": float(penalty_alpha),
            "w_skipped": float(w_skipped),
            "w_invalid": float(w_invalid),
            "w_delayed": float(w_delayed),
        }
        self.epoch = 0

        self.stakeholders: Dict[str, Stakeholder] = {}
        self.mains: Dict[str, MainAddress] = {}
        self.overflows: Dict[str, OverflowNode] = {}

        self.pending_ops: List[Dict] = []

        self.history: List[Dict] = []

        self.weights_for_epoch: Dict[str, float] = {}
        self.weights_next_epoch: Dict[str, float] = {}

        self.current_schedule: List[str] = []
        self.next_schedule: List[str] = []

        self.current_seed: bytes = b""
        self.next_seed: bytes = b""

        self._slot_outcomes_current: List[str] = []

        self._bootstrap_done = False
        self._initialized_weights = False

    # ---------------------------
    # Registrazione / indexing
    # ---------------------------
    def register_stakeholder(self, st: Stakeholder):
        self.stakeholders[st.address] = st

    def register_main(self, m: MainAddress):
        self.mains[m.address] = m

    def index_overflow(self, node: OverflowNode):
        if node.main_id not in self.mains:
            raise ValueError("main_id non registrato")
        self.overflows[node.address] = node

    # ---------------------------
    # Operazioni differite (EEP window)
    # ---------------------------
    def queue_bet(self, from_addr: str, to_main_id: str, amount: float, when_slot: int):
        if to_main_id not in self.mains:
            raise ValueError(f"Main non registrato: {to_main_id}")
        if from_addr not in self.stakeholders and from_addr != "_self_":
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        if amount < MIN_BET_AMOUNT:
            raise ValueError(f"Importo minimo di puntata è {MIN_BET_AMOUNT}, got {amount}")
        self.pending_ops.append({
            "kind": "STAKE",
            "from": from_addr,
            "to": to_main_id,
            "amount": float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_stake_undo(self, from_addr: str, when_slot: int):
        if from_addr not in self.stakeholders:
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        self.pending_ops.append({
            "kind": "STAKE_UNDO",
            "from": from_addr,
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_self_unbet(self, main_id: str, when_slot: int, amount: Optional[float] = None):
        if main_id not in self.mains:
            raise ValueError(f"Main non registrato: {main_id}")
        self.pending_ops.append({
            "kind": "SELF_UNBET",
            "to": main_id,
            "amount": None if amount is None else float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    # ---------------------------
    # Applica op che maturano ora (per preparare t+1)
    # ---------------------------
    def _apply_ops_at_current_eep(self):
        eep = self.cfg["eep_slot"]
        eligible, keep = [], []
        for op in self.pending_ops:
            if (op["epoch"] == self.epoch and op["slot"] <= eep) or \
               (op["epoch"] == self.epoch - 1 and op["slot"] > eep):
                eligible.append(op)
            else:
                keep.append(op)

        def op_key(op):
            return (
                0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                op.get("epoch", -1),
                op.get("slot", -1),
                op.get("from", ""),
                op.get("to", "")
            )
        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.stakeholders.get(deleg_addr)
                if not st:
                    continue
                for m in self.mains.values():
                    cur = m.delegates.get(deleg_addr, 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(deleg_addr, None)

            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if not to_main:
                    continue
                amount = float(op["amount"])
                if amount < MIN_BET_AMOUNT:
                    continue
                from_addr = op["from"]
                if from_addr == "_self_":
                    if amount <= to_main.green:
                        to_main.green -= amount
                        to_main.bet("_self_", amount)
                else:
                    st = self.stakeholders.get(from_addr)
                    if st and amount <= st.green:
                        st.green -= amount
                        to_main.bet(from_addr, amount)

            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amt = op.get("amount", None)
                    to_main.self_unbet(amt)

        self.pending_ops = keep

    # ---------------------------
    # Snapshot EEP (fotografa i main)
    # ---------------------------
    def _snapshot_all_mains_at_eep(self):
        for m in self.mains.values():
            m.snapshot_at_eep()

    # ---------------------------
    # Helpers hashing/seed
    # ---------------------------
    @staticmethod
    def _hash_bytes(*parts: bytes) -> bytes:
        h = hashlib.sha256()
        for p in parts:
            h.update(p)
        return h.digest()

    def _epoch_seed_str(self) -> str:
        return f"takamaka-seed:{self.cfg['seed']}:{self.epoch}"

    # ---------------------------
    # Pesi effettivi dallo stato corrente (stake base → attenuata da penalty slots)
    # ---------------------------
    def _compute_effective_weights_now(self) -> Dict[str, float]:
        weights: Dict[str, float] = {}
        alpha = self.cfg["penalty_alpha"]
        for node in self.overflows.values():
            main = self.mains[node.main_id]
            base = max(0.0, main.active_stake * max(0.0, node.base_stake_share))
            resid = getattr(node, "penalty_slots_residual", 0)
            eff = base / (1.0 + alpha * max(0, int(resid)))
            node.penalty_score = alpha * max(0, int(resid))
            weights[node.address] = eff
        return weights

    # ---------------------------
    # INT_ADDR = ToPositiveBigInt(HashByte(ADDR)) e intervalli stake ordinati per INT_ADDR
    #   FIX: quantizzazione fixed-point intera (troncamento) + scala grande
    # ---------------------------
    @staticmethod
    def _int_addr(addr: str) -> int:
        return int.from_bytes(hashlib.sha256(addr.encode()).digest(), "big", signed=False)

    def _build_intervals_from_weights_vrf(self, weights: Dict[str, float]) -> Tuple[List[str], List[int], int]:
        scale = int(self.cfg["stake_scale_int"])
        items = []
        for addr, w in weights.items():
            w = 0.0 if w is None else float(w)
            wint = int(w * scale)
            if wint <= 0:
                continue
            items.append((addr, self._int_addr(addr), wint))
        items.sort(key=lambda x: x[1])

        addresses, prefix_ends = [], []
        acc = 0
        for addr, _, wint in items:
            addresses.append(addr)
            acc += wint
            prefix_ends.append(acc)

        total_int = acc
        return addresses, prefix_ends, total_int

    # ---------------------------
    # Hash iterato del seed (i+1 volte)
    # ---------------------------
    @staticmethod
    def _iter_hash(seed: bytes, n: int) -> bytes:
        x = seed
        for _ in range(n):
            x = hashlib.sha256(x).digest()
        return x

    # ---------------------------
    # VRF per-slot (PRF deterministica) con mapping stabile dei bordi
    # ---------------------------
    def _vrf_schedule_exact(self, seed: bytes, weights: Dict[str, float]) -> Tuple[Dict[str, int], List[str]]:
        S = self.cfg["slots_per_epoch"]
        addrs, prefix_ends, total_int = self._build_intervals_from_weights_vrf(weights)
        if total_int <= 0 or not addrs:
            return {addr: 0 for addr in self.overflows}, []

        counts = {addr: 0 for addr in self.overflows}
        schedule: List[str] = []

        for i in range(S):
            ri = self._iter_hash(seed, i + 1)
            ticket = int.from_bytes(ri, "big") % total_int
            j = bisect.bisect_left(prefix_ends, ticket)
            if j >= len(addrs):
                j = len(addrs) - 1
            addr = addrs[j]
            schedule.append(addr)
            counts[addr] = counts.get(addr, 0) + 1

        return counts, schedule

    # ---------------------------
    # Penalità e reward (con penalty slots + recovery per slot)
    # ---------------------------
    def _apply_penalties_and_rewards(self, per_node_outcomes: Dict[str, Dict[str, int]]):
        wS = self.cfg["w_skipped"]
        wI = self.cfg["w_invalid"]
        wD = self.cfg["w_delayed"]
        PF = self.cfg["penalty_factor"]
        RF = self.cfg["recovery_factor"]
        tol_ratio = self.cfg["tolerated_bad_ratio"]

        for addr, outcome in per_node_outcomes.items():
            node = self.overflows[addr]
            main = self.mains[node.main_id]

            assigned = max(1, node.assigned_slots)
            ok = int(outcome.get("ok", 0))
            skipped = int(outcome.get("skipped", 0))
            invalid = int(outcome.get("invalid", 0))
            delayed = int(outcome.get("delayed", 0))

            tol = int(tol_ratio * assigned)
            bad_weighted = (wS * skipped) + (wI * invalid) + (wD * delayed)
            gained = max(0.0, bad_weighted - tol) * PF
            gained_i = int(math.ceil(gained))

            resid_prev = getattr(node, "penalty_slots_residual", 0)
            resid_next = max(0, int(resid_prev) - int(RF * ok)) + gained_i
            node.penalty_slots_residual = int(resid_next)

            node.penalty_score = self.cfg["penalty_alpha"] * node.penalty_slots_residual

            spiv = self.cfg["slash_per_invalid"]
            if spiv > 0.0 and invalid > 0:
                node.slash_green(invalid * spiv)

            reward = self.cfg["reward_per_block"] * ok
            main.credit_reward(reward)

    # ---------------------------
    # Seed next dall'EEP (back-search se slot EEP è saltato)
    # ---------------------------
    def _compute_next_seed_from_eep_block(self) -> bytes:
        eep_idx = max(0, self.cfg["eep_slot"] - 1)

        chosen_slot = None
        if self._slot_outcomes_current:
            i = min(eep_idx, len(self._slot_outcomes_current) - 1)
            while i >= 0:
                if self._slot_outcomes_current[i] == "ok":
                    chosen_slot = i
                    break
                i -= 1

        if chosen_slot is None:
            return hashlib.sha256(b"fallback|seed|" + str(self.epoch).encode()).digest()

        producer_addr = self.current_schedule[chosen_slot]
        block_message = (b"|epoch|" + self.epoch.to_bytes(8, "big") +
                         b"|slot|" + chosen_slot.to_bytes(8, "big") +
                         b"|producer|" + producer_addr.encode())
        vrf_seed = hashlib.sha256(block_message).digest()
        return vrf_seed

    # ---------------------------
    # Prepara stato per t+1: (EEP) op+snapshot → pesi ; seed ; NEXT schedule
    # ---------------------------
    def _prepare_next_epoch_state(self):
        self._apply_ops_at_current_eep()
        self._snapshot_all_mains_at_eep()
        self.weights_next_epoch = self._compute_effective_weights_now()

        self.next_seed = self._compute_next_seed_from_eep_block()

        _, self.next_schedule = self._vrf_schedule_exact(self.next_seed, self.weights_next_epoch)

    # ---------------------------
    # Bootstrap epoch 0: CESD0 + NESD0 uguali
    # ---------------------------
    def _bootstrap_epoch0_if_needed(self):
        if self.epoch != 0 or self._bootstrap_done:
            return

        eligible, keep = [], []
        for op in self.pending_ops:
            if op["epoch"] == 0 and op["slot"] == 0:
                eligible.append(op)
            else:
                keep.append(op)

        def op_key(op):
            return (0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                    op.get("epoch", -1), op.get("slot", -1), op.get("from", ""), op.get("to", ""))
        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.stakeholders.get(deleg_addr)
                if st:
                    for m in self.mains.values():
                        cur = m.delegates.get(deleg_addr, 0.0)
                        if cur > 0:
                            st.green += cur
                            m.delegates.pop(deleg_addr, None)
            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amount = float(op["amount"])
                    if amount >= MIN_BET_AMOUNT:
                        from_addr = op["from"]
                        if from_addr == "_self_":
                            if amount <= to_main.green:
                                to_main.green -= amount
                                to_main.bet("_self_", amount)
                        else:
                            st = self.stakeholders.get(from_addr)
                            if st and amount <= st.green:
                                st.green -= amount
                                to_main.bet(from_addr, amount)
            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amt = op.get("amount", None)
                    to_main.self_unbet(amt)

        self.pending_ops = keep

        self._snapshot_all_mains_at_eep()
        self.weights_for_epoch = self._compute_effective_weights_now()

        self.current_seed = hashlib.sha256(b"bootstrap-seed|" + str(self.cfg['seed']).encode()).digest()

        _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

        self.next_seed = self.current_seed
        self.next_schedule = list(self.current_schedule)

        self._initialized_weights = True
        self._bootstrap_done = True

    # ---------------------------
    # Simulazione epoch
    # ---------------------------
    def simulate_one_epoch(self) -> Dict:
        """
        A) (t=0) bootstrap (CESD0=NESD0). Per t>0: current_schedule già impostata (da NEXT precedente).
        B) Simula gli slot dell'epoca t seguendo current_schedule → outcomes, penalità/reward.
        C) All'EEP di t: op+snapshot, fissa next_seed e costruisci next_schedule.
        D) KPI e rollover: aggiorna pesi/schedule e passa all'epoca successiva.
        """
        if self.epoch == 0 and not self._bootstrap_done:
            self._bootstrap_epoch0_if_needed()
        elif not self.current_schedule:
            if not self._initialized_weights:
                self.weights_for_epoch = self._compute_effective_weights_now()
                self._initialized_weights = True
            self.current_seed = hashlib.sha256(self._epoch_seed_str().encode() + b"|current-seed|").digest()
            _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

        for addr, node in self.overflows.items():
            node.effective_stake = float(self.weights_for_epoch.get(addr, 0.0))

        per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
        self._slot_outcomes_current = []

        rng_slots = _rnd.Random(self.cfg["seed"] * 7919 + self.epoch)
        for addr in self.current_schedule:
            outcome = self.overflows[addr].behavior_outcome(rng_slots)
            per_node_outcomes[addr][outcome] += 1
            self._slot_outcomes_current.append(outcome)

        slots_per_node = {addr: 0 for addr in self.overflows}
        for addr in self.current_schedule:
            slots_per_node[addr] = slots_per_node.get(addr, 0) + 1
        for addr, node in self.overflows.items():
            node.assigned_slots = slots_per_node.get(addr, 0)

        self._apply_penalties_and_rewards(per_node_outcomes)

        self._prepare_next_epoch_state()

        total_effective_stake = sum(self.weights_for_epoch.values())
        total_slots_epoch = int(self.cfg["slots_per_epoch"])
        kpi = {
            "epoch": self.epoch,
            "slots_per_node": dict(slots_per_node),
            "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
            "penalty_slots_residual": {addr: getattr(self.overflows[addr], "penalty_slots_residual", 0)
                                       for addr in self.overflows},
            "penalties_alpha_scaled": {addr: self.overflows[addr].penalty_score for addr in self.overflows},
            "effective_stake_per_overflow": {addr: self.overflows[addr].effective_stake for addr in self.overflows},
            "total_effective_stake": total_effective_stake,
            "total_slots": total_slots_epoch,
            "seed_current_hex": self.current_seed.hex(),
            "seed_next_hex": self.next_seed.hex(),
        }
        self.history.append(kpi)

        self.weights_for_epoch = dict(self.weights_next_epoch)
        self.current_seed = self.next_seed
        self.current_schedule = list(self.next_schedule)

        self.epoch += 1
        return kpi

    # ---------------------------
    # Report epoch → DataFrame
    # ---------------------------
    def epoch_summary_df(self, kpi: Dict) -> pd.DataFrame:
        total_effective = kpi.get("total_effective_stake", 0.0)
        total_slots_epoch = int(kpi.get("total_slots", 0))
        rows = []

        for addr, slots in kpi["slots_per_node"].items():
            o = kpi["per_node_outcomes"].get(addr, {})
            node = self.overflows[addr]

            ok = int(o.get("ok", 0))
            skipped = int(o.get("skipped", 0))
            invalid = int(o.get("invalid", 0))
            delayed = int(o.get("delayed", 0))
            bad_abs = skipped + invalid + delayed

            stake_share_pct = (node.effective_stake / total_effective * 100.0) if total_effective > 0 else 0.0
            delivered_slots_share_pct = (ok / total_slots_epoch * 100.0) if total_slots_epoch > 0 else 0.0
            bad_slots_pct_node = (bad_abs / slots * 100.0) if slots > 0 else 0.0

            rows.append({
                "epoch": kpi["epoch"],
                "overflow": addr,
                "main": node.main_id,
                "assigned_slots": int(slots),

                "ok": ok,
                "skipped": skipped,
                "invalid": invalid,
                "delayed": delayed,

                "penalty_slots_residual": getattr(node, "penalty_slots_residual", 0),
                "penalty_score": node.penalty_score,

                "effective_stake": node.effective_stake,
                "stake_share_pct": round(stake_share_pct, 2),
                "delivered_slots_share_pct": round(delivered_slots_share_pct, 2),
                "bad_slots_abs": int(bad_abs),
                "bad_slots_pct_node": round(bad_slots_pct_node, 2),
            })

        return pd.DataFrame(rows)


# Simulations

## Simulation 1

This simulation runs for 20 epochs.
- All nodes begin with stake of 100
- All nodes behave correctly, meaning that every assigned slot is validated as “ok.”

The final report is saved to ./data/simulation_1.csv.

In [12]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,   # 2400
    slot_duration_sec=SLOT_DURATION_SEC,
    seed=RNG_SEED
)

mA = MainAddress(address="MAIN_A", green=0.0)
mB = MainAddress(address="MAIN_B", green=0.0)
mC = MainAddress(address="MAIN_C", green=0.0)

sim.register_main(mA)
sim.register_main(mB)
sim.register_main(mC)

oA = OverflowNode(address="OF_A", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)
oB = OverflowNode(address="OF_B", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)
oC = OverflowNode(address="OF_C", green=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0)

mA.register_overflow(oA); sim.index_overflow(oA)
mB.register_overflow(oB); sim.index_overflow(oB)
mC.register_overflow(oC); sim.index_overflow(oC)

sA = Stakeholder(address="STK_A", green=100.0)
sB = Stakeholder(address="STK_B", green=100.0)
sC = Stakeholder(address="STK_C", green=100.0)

sim.register_stakeholder(sA)
sim.register_stakeholder(sB)
sim.register_stakeholder(sC)

sim.queue_bet(from_addr="STK_A", to_main_id="MAIN_A", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_B", to_main_id="MAIN_B", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_C", to_main_id="MAIN_C", amount=100.0, when_slot=0)

kpis = []
dfs = []

for _ in range(20):
    kpi = sim.simulate_one_epoch()
    kpis.append(kpi)
    dfs.append(sim.epoch_summary_df(kpi))

summary = pd.concat(dfs, ignore_index=True)
out_path = "./data/simulation_ok.csv"
summary.to_csv(out_path, index=False)


## Simulation 2

- Identical setup to Simulation 1.
- The only change is that Node 1 behaves maliciously with error probabilities p_skip = 0.02 and p_invalid = 0.01 (others remain honest). 

The final report is saved to ./data/simulation_2.csv.

In [ ]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    slot_duration_sec=SLOT_DURATION_SEC,
    seed=RNG_SEED
)

mA = MainAddress(address="MAIN_A", green=0.0)
mB = MainAddress(address="MAIN_B", green=0.0)
mC = MainAddress(address="MAIN_C", green=0.0)
sim.register_main(mA); sim.register_main(mB); sim.register_main(mC)

oA = OverflowNode(address="OF_A", green=0.0, p_skip=0.02, p_invalid=0.01, p_delayed=0.0)
oB = OverflowNode(address="OF_B", green=0.0, p_skip=0.0,  p_invalid=0.0,  p_delayed=0.0)
oC = OverflowNode(address="OF_C", green=0.0, p_skip=0.0,  p_invalid=0.0,  p_delayed=0.0)
mA.register_overflow(oA); sim.index_overflow(oA)
mB.register_overflow(oB); sim.index_overflow(oB)
mC.register_overflow(oC); sim.index_overflow(oC)

sA = Stakeholder(address="STK_A", green=100.0)
sB = Stakeholder(address="STK_B", green=100.0)
sC = Stakeholder(address="STK_C", green=100.0)
sim.register_stakeholder(sA); sim.register_stakeholder(sB); sim.register_stakeholder(sC)

sim.queue_bet(from_addr="STK_A", to_main_id="MAIN_A", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_B", to_main_id="MAIN_B", amount=100.0, when_slot=0)
sim.queue_bet(from_addr="STK_C", to_main_id="MAIN_C", amount=100.0, when_slot=0)

dfs = []
for _ in range(20):
    kpi = sim.simulate_one_epoch()
    dfs.append(sim.epoch_summary_df(kpi))

import pandas as pd
summary_faulty = pd.concat(dfs, ignore_index=True)

cols_order = [
    "epoch","overflow","main","assigned_slots",
    "ok","skipped","invalid","delayed",
    "penalty_slots_residual","penalty_score",
    "effective_stake","stake_share_pct","delivered_slots_share_pct",
    "bad_slots_abs","bad_slots_pct_node"
]

for c in cols_order:
    if c not in summary_faulty.columns:
        summary_faulty[c] = 0

summary_faulty = summary_faulty[cols_order]

out_path = "data/simulation_faulty.csv"
summary_faulty.to_csv(out_path, index=False)